# 构造url

In [1]:
url_template = "https://weibo.cn/u/{user_id}?page={page}"

user_id = "1682207150"


In [114]:
url_list = []

for page in range(1, 501):
    url = url_template.format(user_id=user_id, page=page)
    url_list.append(url)
    
url_list

['https://weibo.cn/u/1682207150?page=1',
 'https://weibo.cn/u/1682207150?page=2',
 'https://weibo.cn/u/1682207150?page=3',
 'https://weibo.cn/u/1682207150?page=4',
 'https://weibo.cn/u/1682207150?page=5',
 'https://weibo.cn/u/1682207150?page=6',
 'https://weibo.cn/u/1682207150?page=7',
 'https://weibo.cn/u/1682207150?page=8',
 'https://weibo.cn/u/1682207150?page=9',
 'https://weibo.cn/u/1682207150?page=10',
 'https://weibo.cn/u/1682207150?page=11',
 'https://weibo.cn/u/1682207150?page=12',
 'https://weibo.cn/u/1682207150?page=13',
 'https://weibo.cn/u/1682207150?page=14',
 'https://weibo.cn/u/1682207150?page=15',
 'https://weibo.cn/u/1682207150?page=16',
 'https://weibo.cn/u/1682207150?page=17',
 'https://weibo.cn/u/1682207150?page=18',
 'https://weibo.cn/u/1682207150?page=19',
 'https://weibo.cn/u/1682207150?page=20',
 'https://weibo.cn/u/1682207150?page=21',
 'https://weibo.cn/u/1682207150?page=22',
 'https://weibo.cn/u/1682207150?page=23',
 'https://weibo.cn/u/1682207150?page=24',
 

# 函数：请求数据

In [3]:
import requests

def get_html(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0",
           "cookie":'SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WWreg3M85waDCfwhLTHsUGS5JpX5KMhUgL.FoMX1h-p1hzfe0q2dJLoIp7LxKML1KBLBKnLxKqL1hnLBoMceh5EShe4S0nf; ALF=1737460694; SUB=_2A25KY46GDeRhGeFK41cQ-CzJyDqIHXVpAI5OrDV8PUJbkNAbLRPNkW1NQsp0yEnMqGgG3mpiBu9fpzLIyGAoKnTl'}

    response = requests.get(url, headers=headers)
    
    html = response.text[38:]
    
    return html


# 函数：解析数据

In [109]:
from pyquery import PyQuery as pq
import re

def extract_weibo_list(html):

    doc = pq(html)

    weibo_items = doc('div.c').eq(slice(1, -1)) # 选择所有 div.c 元素，排除第一个和最后一个


    weibo_list =[]

    for item in weibo_items.items():
        content = item('div span.ctt').text() # 获取微博内容
        full_text_url = item('div span.ctt a:last-child').attr('href') # 获取微博全文链接

        
        repost_source_name = item('div:nth-child(1) span.cmt a').text() # 获取转发来源名称
        repost_source_url = item('div:nth-child(1) span.cmt a').attr("href") # 获取转发来源链接
        repost_reason = item('div:nth-child(2)').text().partition('\n')[0] # 获取转发理由
        
        # # 获取所有的 a 标签
        a_tags = item('a').items()
        list_a_tags = list(a_tags) # 转化为列表
        
        like_count_raw = list_a_tags[-4].text() # 倒数第四个 a 标签
        like_count = re.search(r'\[(\d+)\]', like_count_raw).group(1)
        
        repost_count_raw = list_a_tags[-3].text() # 倒数第三个 a 标签
        repost_count = re.search(r'\[(\d+)\]', repost_count_raw).group(1)
        
        comment_count_raw = list_a_tags[-2].text() # 倒数第二个 a 标签
        comment_count = re.search(r'\[(\d+)\]', comment_count_raw).group(1)


        time = item('div span.ct').text().split('\xa0')[0] # 获取微博发布时间
        
        
        weibo_dict = {
            "content": content,
            "full_text_url": full_text_url,
            "repost_source_name": repost_source_name,
            "repost_source_url": repost_source_url,
            "repost_reason": repost_reason,
            "like_count": like_count,
            "repost_count": repost_count,
            "comment_count": comment_count,
            "time": time
        }   
    
    
        weibo_list.append(weibo_dict)
        
    return weibo_list

# 测试

In [111]:
url = url_list[0]  # 取第一个 url 进行示例
url

'https://weibo.cn/u/1682207150?page=1'

In [112]:
html = get_html(url)
html

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd"><html xmlns="http://www.w3.org/1999/xhtml"><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8" /><meta http-equiv="Cache-Control" content="no-cache"/><meta id="viewport" name="viewport" content="width=device-width,initial-scale=1.0,minimum-scale=1.0, maximum-scale=2.0" /><link rel="shortcut icon" type="image/x-icon" href="https://weibo.cn/favicon.ico"><link rel="icon" sizes="any" mask href="https://h5.sinaimg.cn/upload/2015/05/15/28/WeiboLogoCh.svg" color="black"><meta name="MobileOptimized" content="240"/><title>南方日报的微博</title><style type="text/css" id="internalStyle">html,body,p,form,div,table,textarea,input,span,select{font-size:12px;word-wrap:break-word;}body{background:#F8F9F9;color:#000;padding:1px;margin:1px;}table,tr,td{border-width:0px;margin:0px;padding:0px;}form{margin:0px;padding:0px;border:0px;}textarea{border:1px solid #96c1e6}textarea{width:9

In [113]:
extract_weibo_list(html)

[{'content': '【#专家提醒黑色染发剂最危险#】年过50的李女士近十年频繁染发，近段时间，李女士因身体疲乏、皮肤暗淡而到了医院就诊，结果被查出肝硬化中晚期。医生分析，李女士的肝硬化与长期频繁染发有关。\n\n来自美国相关机构的研究发现，使用永久染发剂与卵巢病、皮肤癌、淋巴瘤乳腺癌等癌症风险增加有关。黑色染...全文',
  'full_text_url': '/comment/P6H5PikLZ?ckAll=1',
  'repost_source_name': '',
  'repost_source_url': None,
  'repost_reason': '',
  'like_count': '4',
  'repost_count': '0',
  'comment_count': '1',
  'time': '12分钟前'},
 {'content': '#大疆拟建250米超高层总部大楼#【大疆拟深圳湾超总拿地，建250米+超高层总部大楼！】12月25日，深圳市政府官网发布《智能航空系统产业生态全球总部项目遴选方案》显示，宗地位于深圳湾超级总部基地DU01-02地块，用地面积15658平方米，总建筑面积188000㎡。\n\n目前意向用地单位为深圳市大疆创新科技有限...全文',
  'full_text_url': '/comment/P6GVqANQJ?ckAll=1',
  'repost_source_name': '',
  'repost_source_url': None,
  'repost_reason': '原图',
  'like_count': '2',
  'repost_count': '0',
  'comment_count': '0',
  'time': '37分钟前'},
 {'content': '#鱿鱼游戏3明年开播#【小调查】《鱿鱼游戏》第二季今日上线，第一季的《鱿鱼游戏》你看过吗？ 你看过第一季的《鱿鱼游戏》吗？',
  'full_text_url': 'https://vote.weibo.com/h5/index/index?vote_id=2024_4022115_-_2890fd',
  'repost_source_name': '',
  'repost_source_url'

# 正式抓取

In [115]:
import random
import time

all_weibo_list = [] # 存储所有微博的列表

for url in url_list:

    print(f"正在抓取：{url}")
    html = get_html(url) # 获取 HTML 内容

    weibo_list = extract_weibo_list(html) # 提取微博列表
    
    wait_time = random.randint(3, 5) # 生成 3 到 5 秒的随机浮点数
    print(f"等待{wait_time}秒")
    time.sleep(wait_time)   # 暂停指定的秒数
    print("-"*50)
        
    all_weibo_list.extend(weibo_list) # 将当前页面的微博列表添加到总的列表中

print("抓取完成")


正在抓取：https://weibo.cn/u/1682207150?page=1
等待5秒
--------------------------------------------------
正在抓取：https://weibo.cn/u/1682207150?page=2
等待4秒
--------------------------------------------------
正在抓取：https://weibo.cn/u/1682207150?page=3
等待5秒
--------------------------------------------------
正在抓取：https://weibo.cn/u/1682207150?page=4
等待5秒
--------------------------------------------------
正在抓取：https://weibo.cn/u/1682207150?page=5
等待5秒
--------------------------------------------------
正在抓取：https://weibo.cn/u/1682207150?page=6
等待3秒
--------------------------------------------------
正在抓取：https://weibo.cn/u/1682207150?page=7
等待3秒
--------------------------------------------------
正在抓取：https://weibo.cn/u/1682207150?page=8
等待4秒
--------------------------------------------------
正在抓取：https://weibo.cn/u/1682207150?page=9
等待5秒
--------------------------------------------------
正在抓取：https://weibo.cn/u/1682207150?page=10
等待4秒
--------------------------------------------------
正在抓取：https://weibo.

# 检查数据

In [116]:
all_weibo_list

[{'content': '【#专家提醒黑色染发剂最危险#】年过50的李女士近十年频繁染发，近段时间，李女士因身体疲乏、皮肤暗淡而到了医院就诊，结果被查出肝硬化中晚期。医生分析，李女士的肝硬化与长期频繁染发有关。\n\n来自美国相关机构的研究发现，使用永久染发剂与卵巢病、皮肤癌、淋巴瘤乳腺癌等癌症风险增加有关。黑色染...全文',
  'full_text_url': '/comment/P6H5PikLZ?ckAll=1',
  'repost_source_name': '',
  'repost_source_url': None,
  'repost_reason': '',
  'like_count': '5',
  'repost_count': '0',
  'comment_count': '1',
  'time': '17分钟前'},
 {'content': '#大疆拟建250米超高层总部大楼#【大疆拟深圳湾超总拿地，建250米+超高层总部大楼！】12月25日，深圳市政府官网发布《智能航空系统产业生态全球总部项目遴选方案》显示，宗地位于深圳湾超级总部基地DU01-02地块，用地面积15658平方米，总建筑面积188000㎡。\n\n目前意向用地单位为深圳市大疆创新科技有限...全文',
  'full_text_url': '/comment/P6GVqANQJ?ckAll=1',
  'repost_source_name': '',
  'repost_source_url': None,
  'repost_reason': '原图',
  'like_count': '2',
  'repost_count': '0',
  'comment_count': '0',
  'time': '42分钟前'},
 {'content': '#鱿鱼游戏3明年开播#【小调查】《鱿鱼游戏》第二季今日上线，第一季的《鱿鱼游戏》你看过吗？ 你看过第一季的《鱿鱼游戏》吗？',
  'full_text_url': 'https://vote.weibo.com/h5/index/index?vote_id=2024_4022115_-_2890fd',
  'repost_source_name': '',
  'repost_source_url'

In [117]:
len(all_weibo_list)

4891

# 导出数据

In [118]:
import pandas as pd

df = pd.DataFrame(all_weibo_list)

df.to_excel('南方日报_微博用户.xlsx', index=False)